In [ ]:
import json
import matplotlib.pyplot as plt
import random
import requests
import time
import uuid
import datetime

In [ ]:
# Replace 'personalization_base_url' and 'resource_key' with your valid endpoint values:
personalization_base_url = "<your-endpoint>"
resource_key = "<your-resource-key>"

In [ ]:
# Print out current datetime:
def currentDateTime():
    currentDT = datetime.datetime.now()
    print (str(currentDT))

# Ititialize variable for model's last modified date:
modelLastModified = ""

def get_last_updated(currentModifiedDate):

    print('-----checking model')

    # Get Model Properties:
    response = requests.get(personalization_model_properties_url, headers = headers, params = None)

    print(response)
    print(response.json())

    # Get lastModifiedTime:
    lastModifiedTime = json.dumps(response.json()["lastModifiedTime"])

    if (currentModifiedDate != lastModifiedTime):
        currentModifiedDate = lastModifiedTime
        print(f'-----model updated: {lastModifiedTime}')

In [ ]:
def get_service_settings():

    print('-----checking service settings')

    # Get Learning Policy:
    response = requests.get(personalization_model_policy_url, headers = headers, params = None)

    print(response)
    print(response.json())

    # Get Service Settings:
    response = requests.get(personalization_service_configuration_url, headers = headers, params = None)

    print(response)
    print(response.json())

In [ ]:
# Build URLs:
personalization_rank_url = personalization_base_url + "personalizer/v1.0/rank"
personalization_reward_url = personalization_base_url + "personalizer/v1.0/events/" #add "{eventId}/reward"
personalization_model_properties_url = personalization_base_url + "personalizer/v1.0/model/properties"
personalization_model_policy_url = personalization_base_url + "personalizer/v1.0/configurations/policy"
personalization_service_configuration_url = personalization_base_url + "personalizer/v1.0/configurations/service"

headers = {'Ocp-Apim-Subscription-Key' : resource_key, 'Content-Type': 'application/json'}

# Context:
users = "users.json"

# Action Features:
coffee = "coffee.json"

# Empty JSON for Rank Request:
requestpath = "example-rankrequest.json"

# Initialize Random:
random.seed(time.time())

userpref = None
rankactionsjsonobj = None
actionfeaturesobj = None

with open(users) as handle:
    userpref = json.loads(handle.read())

with open(coffee) as handle:
    actionfeaturesobj = json.loads(handle.read())

with open(requestpath) as handle:
    rankactionsjsonobj = json.loads(handle.read())

get_last_updated(modelLastModified)
get_service_settings()

print(f'User count {len(userpref)}')
print(f'Coffee count {len(actionfeaturesobj)}')

In [ ]:
def add_event_id(rankjsonobj):
    eventid = uuid.uuid4().hex
    rankjsonobj["eventId"] = eventid
    return eventid

In [ ]:
def add_random_user_and_contextfeatures(namesoption, weatheropt, timeofdayopt, rankjsonobj):
    name = namesoption[random.randint(0,3)]
    weather = weatheropt[random.randint(0,2)]
    timeofday = timeofdayopt[random.randint(0,2)]
    rankjsonobj['contextFeatures'] = [{'timeofday': timeofday, 'weather': weather, 'name': name}]
    return [name, weather, timeofday]

In [ ]:
def add_action_features(rankjsonobj):
    rankjsonobj["actions"] = actionfeaturesobj

In [ ]:
def get_reward_from_simulated_data(name, weather, timeofday, prediction):
    if(userpref[name][weather][timeofday] == str(prediction)):
        return 1
    return 0

In [ ]:
def iterations(n, modelCheck, jsonFormat):

    i = 1

    # Default reward value, assumes failed prediction:
    reward = 0

    # Print out dateTime:
    currentDateTime()

    # Collect results to aggregate in graph:
    total = 0
    rewards = []
    count = []

    # Default list of user, weather, time of day:
    namesopt = ['Alice', 'Bob', 'Cathy', 'Dave']
    weatheropt = ['Sunny', 'Rainy', 'Snowy']
    timeofdayopt = ['Morning', 'Afternoon', 'Evening']

    while(i <= n):

        # Create unique id to associate with an event:
        eventid = add_event_id(jsonFormat)

        # Generate a Random Sample:
        [name, weather, timeofday] = add_random_user_and_contextfeatures(namesopt, weatheropt, timeofdayopt, jsonFormat)

        # Add action features to rank:
        add_action_features(jsonFormat)

        # Show JSON to send to Rank:
        print('To: ', jsonFormat)

        # Choose an action - get prediction from Personalizer:
        response = requests.post(personalization_rank_url, headers = headers, params = None, json = jsonFormat)

        # Show Rank Prediction:
        print ('From: ',response.json())

        # Compare personalization service recommendation with the simulated data to generate a reward value:
        prediction = json.dumps(response.json()["rewardActionId"]).replace('"','')
        reward = get_reward_from_simulated_data(name, weather, timeofday, prediction)*10

        # Show Result for Iteration:
        print(f'   {i} {currentDateTime()} {name} {weather} {timeofday} {prediction} {reward}')

        # Send the reward to the service:
        response = requests.post(personalization_reward_url + eventid + "/reward", headers = headers, params= None, json = { "value" : reward })

        # For every N rank requests, compute total correct  total:
        total =  total + reward

        # Every N iteration, get last updated model date and time:
        if(i % modelCheck == 0):

            print("**** 10% of loop found")

            get_last_updated(modelLastModified)

        # Aggregate so chart is easier to read:
        if(i % 10 == 0):
            rewards.append( total)
            count.append(i)
            total = 0

        i = i + 1

    # Print out dateTime:
    currentDateTime()

    return [count, rewards]

In [ ]:
# Max Iterations:
num_requests = 1500

# Check last mod date N% of time (currently 10%):
lastModCheck = int(num_requests * .10)

jsonTemplate = rankactionsjsonobj

# Main Iterations:
[count, rewards] = iterations(num_requests, lastModCheck, jsonTemplate)

In [ ]:
def createChart(x, y):
    plt.plot(x, y)
    plt.xlabel("Batch of rank events")
    plt.ylabel("Correct recommendations per batch")
    plt.show()

createChart(count,rewards)